In [3]:
import cx_Oracle
import sqlalchemy as sa
import pandas as pd
import getpass as gp

dsn_tns = cx_Oracle.makedsn('exa02-scan', '1521', service_name='FBODSPRD')
conn = cx_Oracle.connect(user=r'FB006641', password=gp.getpass("Password"), dsn=dsn_tns)

Password········


In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import f1_score
from sklearn import metrics
from pprint import pprint
from subprocess import check_call
from sklearn.tree import export_graphviz
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.tree.export import export_text
from sklearn.tree import _tree
import math
import numpy as np
import featuretools as ft
from fancyimpute import KNN
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.over_sampling import SMOTE

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_sql("SELECT * FROM ASIL_TABLO_CEK WHERE IDENTITYTYPE = 'G' AND PROFITCENTER = 'TC' AND SIMDATE < 20190801",conn)

In [7]:
df["CEKDURUMU"].value_counts()

Tahsil Iade    52718
Karsiliksiz     3559
Name: CEKDURUMU, dtype: int64

In [8]:
df["CEKDURUMU"] = df["CEKDURUMU"].map({"Tahsil Iade":0,"Karsiliksiz":1})

In [10]:
for column in df.columns:
    if df[column].dtype == 'O':
        df = df.drop(column, axis = 1)

df = df.drop(["ID","CUSTOMERNUMBER","BIRTHDATE","SIMDATE","DATE1","AMOUNTP1","KURULUSGECENGUN",
              "TKN","SIM_MONTH","SIM_YEAR"],axis = 1)          

In [ ]:
class_report = {}
dot_file = ""
png_file = ""

threshold = {}
thres = []

In [ ]:
col_names = list(df.drop("CEKDURUMU",axis = 1).columns)

In [ ]:
for column in col_names:
    temp_df = df[[column,"CEKDURUMU"]]
    temp_df = temp_df.dropna()

    X = temp_df.drop("CEKDURUMU",axis=1)
    y = temp_df["CEKDURUMU"]
    
    X_train, X_test, y_train, y_test = train_test_split( 
          X, y, test_size = 0.3, random_state = 100)

    temp_X_train = X_train[column]
    temp_X_test = X_test[column]
    temp_X_train = temp_X_train.dropna()
    temp_X_test = temp_X_test.dropna()

    if column == "KANUNITAKIPMUSTERISIMI" or column == "IZLEMEDEMI":
        dt = DecisionTreeClassifier(max_depth=3, random_state=42)
    else:    
        dt = DecisionTreeClassifier(max_depth=3, random_state=42, min_samples_leaf = 50)
        
    dt.fit(temp_X_train.values.reshape(-1,1), y_train)
    y_pred = dt.predict(temp_X_test.values.reshape(-1,1))
    class_report[column] = classification_report(y_test, y_pred,output_dict = True)
    
        

    dot_file = "dot_gercek/" + column + ".dot"
    export_graphviz(dt, dot_file , class_names=True, filled = True, proportion = True)
    
    png_file = "png_gercek/" + column + ".png"
    check_call(['dot','-Tpng', dot_file, '-o',png_file])
    
    dot_file = ""
    png_file = ""
    
    thres.append(temp_X_train.min())

    for i in sorted([a for a in dt.tree_.threshold if a != -2]): 
        thres.append(i)
    thres.append(temp_X_train.max())
    
    threshold[column] = thres
    thres = []


In [ ]:
gercek = pd.DataFrame(columns = ['Değişken', 'Min Value', 'Max Value', 'Karşılıksız Sayısı' , 'Tahsil/İade Sayısı', 'Toplam Sayı', 'Oran'])

In [ ]:
i = 1


for key in threshold:
    i = 1
    while i < len(threshold[key]):        
        
        gercek = gercek.append({
            "Değişken":key,
            "Max Value":threshold[key][i],
            "Min Value":threshold[key][i-1],
            "Toplam Sayı":df.loc[(df[key] >= threshold[key][i-1]) & 
                         (df[key] <= threshold[key][i]),]["CEKDURUMU"].count(),
            "Karşılıksız Sayısı":df.loc[(df[key] >= threshold[key][i-1]) & 
                         (df[key] <= threshold[key][i]),]["CEKDURUMU"].sum(),
            "Tahsil/İade Sayısı":df.loc[(df[key] >= threshold[key][i-1]) & 
                         (df[key] <= threshold[key][i]),]["CEKDURUMU"].count()
            -df.loc[(df[key] >= threshold[key][i-1]) & 
                         (df[key] <= threshold[key][i]),]["CEKDURUMU"].sum()

        },ignore_index=True)
        i += 1
        
    if len(df.loc[df[key].isnull()]) != 0:
        gercek = gercek.append({
            "Değişken":key,
            "Max Value":"Null",
            "Min Value":"Null",
            "Toplam Sayı":df.loc[df[key].isnull()]["CEKDURUMU"].count(),
            "Karşılıksız Sayısı":df.loc[df[key].isnull()]["CEKDURUMU"].sum(),
            "Tahsil/İade Sayısı":df.loc[df[key].isnull()]["CEKDURUMU"].count()-df.loc[df[key].isnull()]["CEKDURUMU"].sum()

        },ignore_index=True)              
    

In [ ]:
gercek["Oran"] = (gercek["Karşılıksız Sayısı"]/gercek["Toplam Sayı"])*100

gercek["Oran"] = gercek["Oran"].astype("float")

gercek["Oran"] = round(gercek["Oran"],2)


gercek.columns = gercek.columns.str.upper()

In [ ]:
gercek.loc[gercek["DEĞIŞKEN"].str.contains("KREDIBORCORANI")]

In [ ]:
gercek.to_csv("gercek.csv",index= False)

In [ ]:
score = {}

In [ ]:
for clas in class_report:
    score[clas] = class_report[clas]["macro avg"]["f1-score"]

In [ ]:
score

In [ ]:
sorted(score, key=score.get, reverse = True)

In [62]:
bdf = df.dropna()

In [79]:
bdf = df.drop("KURUMSALKKBGECIKMESI",axis = 1)

##bdf = bdf.dropna()

In [80]:
imp_mean = IterativeImputer(random_state=0)
imp = imp_mean.fit_transform(df)

bdf = pd.DataFrame(imp, columns = df.columns)

bdf.loc[bdf["CEKENDEKSI"] > 1000,"CEKENDEKSI"] = 1000
bdf.loc[bdf["KKB"] > 1900,"KKB"] = 1900
bdf.loc[bdf["BBE"] > 60,"BBE"] = 60
bdf = bdf.loc[bdf["AMOUNTTL"] < 500000]

In [11]:
train_cols = list(df)

bdf = pd.DataFrame(KNN(k=7).fit_transform(df))
bdf.columns = train_cols

Imputing row 1/56277 with 0 missing, elapsed time: 1680.658
Imputing row 101/56277 with 0 missing, elapsed time: 1680.867
Imputing row 201/56277 with 1 missing, elapsed time: 1680.913
Imputing row 301/56277 with 1 missing, elapsed time: 1681.008
Imputing row 401/56277 with 1 missing, elapsed time: 1681.084
Imputing row 501/56277 with 1 missing, elapsed time: 1681.142
Imputing row 601/56277 with 0 missing, elapsed time: 1681.183
Imputing row 701/56277 with 0 missing, elapsed time: 1681.242
Imputing row 801/56277 with 2 missing, elapsed time: 1681.299
Imputing row 901/56277 with 2 missing, elapsed time: 1681.362
Imputing row 1001/56277 with 0 missing, elapsed time: 1681.407
Imputing row 1101/56277 with 2 missing, elapsed time: 1681.446
Imputing row 1201/56277 with 1 missing, elapsed time: 1681.496
Imputing row 1301/56277 with 0 missing, elapsed time: 1681.528
Imputing row 1401/56277 with 0 missing, elapsed time: 1681.588
Imputing row 1501/56277 with 1 missing, elapsed time: 1681.624
Impu

Imputing row 13501/56277 with 0 missing, elapsed time: 1686.111
Imputing row 13601/56277 with 0 missing, elapsed time: 1686.146
Imputing row 13701/56277 with 1 missing, elapsed time: 1686.184
Imputing row 13801/56277 with 1 missing, elapsed time: 1686.224
Imputing row 13901/56277 with 0 missing, elapsed time: 1686.268
Imputing row 14001/56277 with 0 missing, elapsed time: 1686.316
Imputing row 14101/56277 with 0 missing, elapsed time: 1686.350
Imputing row 14201/56277 with 0 missing, elapsed time: 1686.390
Imputing row 14301/56277 with 1 missing, elapsed time: 1686.428
Imputing row 14401/56277 with 0 missing, elapsed time: 1686.459
Imputing row 14501/56277 with 0 missing, elapsed time: 1686.484
Imputing row 14601/56277 with 0 missing, elapsed time: 1686.523
Imputing row 14701/56277 with 0 missing, elapsed time: 1686.574
Imputing row 14801/56277 with 0 missing, elapsed time: 1686.614
Imputing row 14901/56277 with 1 missing, elapsed time: 1686.654
Imputing row 15001/56277 with 0 missing,

Imputing row 26301/56277 with 0 missing, elapsed time: 1690.891
Imputing row 26401/56277 with 0 missing, elapsed time: 1690.921
Imputing row 26501/56277 with 0 missing, elapsed time: 1690.964
Imputing row 26601/56277 with 2 missing, elapsed time: 1691.004
Imputing row 26701/56277 with 1 missing, elapsed time: 1691.041
Imputing row 26801/56277 with 2 missing, elapsed time: 1691.084
Imputing row 26901/56277 with 1 missing, elapsed time: 1691.119
Imputing row 27001/56277 with 0 missing, elapsed time: 1691.158
Imputing row 27101/56277 with 1 missing, elapsed time: 1691.199
Imputing row 27201/56277 with 0 missing, elapsed time: 1691.245
Imputing row 27301/56277 with 0 missing, elapsed time: 1691.291
Imputing row 27401/56277 with 2 missing, elapsed time: 1691.335
Imputing row 27501/56277 with 0 missing, elapsed time: 1691.373
Imputing row 27601/56277 with 0 missing, elapsed time: 1691.411
Imputing row 27701/56277 with 1 missing, elapsed time: 1691.443
Imputing row 27801/56277 with 1 missing,

Imputing row 39101/56277 with 1 missing, elapsed time: 1696.460
Imputing row 39201/56277 with 0 missing, elapsed time: 1696.488
Imputing row 39301/56277 with 1 missing, elapsed time: 1696.530
Imputing row 39401/56277 with 0 missing, elapsed time: 1696.585
Imputing row 39501/56277 with 0 missing, elapsed time: 1696.637
Imputing row 39601/56277 with 0 missing, elapsed time: 1696.719
Imputing row 39701/56277 with 0 missing, elapsed time: 1696.762
Imputing row 39801/56277 with 1 missing, elapsed time: 1696.824
Imputing row 39901/56277 with 0 missing, elapsed time: 1696.903
Imputing row 40001/56277 with 0 missing, elapsed time: 1696.964
Imputing row 40101/56277 with 1 missing, elapsed time: 1697.047
Imputing row 40201/56277 with 0 missing, elapsed time: 1697.126
Imputing row 40301/56277 with 0 missing, elapsed time: 1697.165
Imputing row 40401/56277 with 1 missing, elapsed time: 1697.203
Imputing row 40501/56277 with 1 missing, elapsed time: 1697.231
Imputing row 40601/56277 with 1 missing,

Imputing row 51901/56277 with 2 missing, elapsed time: 1701.621
Imputing row 52001/56277 with 3 missing, elapsed time: 1701.659
Imputing row 52101/56277 with 1 missing, elapsed time: 1701.699
Imputing row 52201/56277 with 1 missing, elapsed time: 1701.737
Imputing row 52301/56277 with 0 missing, elapsed time: 1701.777
Imputing row 52401/56277 with 0 missing, elapsed time: 1701.809
Imputing row 52501/56277 with 1 missing, elapsed time: 1701.842
Imputing row 52601/56277 with 0 missing, elapsed time: 1701.869
Imputing row 52701/56277 with 1 missing, elapsed time: 1701.904
Imputing row 52801/56277 with 7 missing, elapsed time: 1701.945
Imputing row 52901/56277 with 19 missing, elapsed time: 1701.986
Imputing row 53001/56277 with 0 missing, elapsed time: 1702.025
Imputing row 53101/56277 with 0 missing, elapsed time: 1702.059
Imputing row 53201/56277 with 0 missing, elapsed time: 1702.099
Imputing row 53301/56277 with 0 missing, elapsed time: 1702.127
Imputing row 53401/56277 with 1 missing

In [76]:
X = bdf.drop("CEKDURUMU",axis=1)
y = bdf["CEKDURUMU"]
    
X_train, X_test, y_train, y_test = train_test_split( 
          X, y, test_size = 0.3, random_state = 100)

In [45]:
cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_resample(X, y)


X_resampled = pd.DataFrame(np.array(X_resampled),columns = list(X))

In [46]:
X_train, X_test, y_train, y_test = train_test_split( 
          X_resampled, y_resampled, test_size = 0.3, random_state = 100)

In [52]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

In [53]:
X_train, X_test, y_train, y_test = train_test_split( 
          X_res, y_res, test_size = 0.3, random_state = 100)

In [82]:
dt = DecisionTreeClassifier(criterion = 'gini', random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)

export_graphviz(dt, out_file="tree.dot", feature_names=X.columns, class_names=True,
                filled = True, proportion= True)

check_call(['dot','-Tpng','tree.dot','-o','OutputFile.png'])


tumdegiskenler = classification_report(y_test, y_pred)

y_pred_prob = dt.predict_proba(X_test)

In [ ]:
temp_list = []

temp_list = y_pred_prob[:,1]

pred_prob = []

In [ ]:
for elem in temp_list:
    if elem > 0.75:
        pred_prob.append(1)
    elif (elem > 0.5) & (elem <=0.75):
        pred_prob.append(0.5)
    else:
        pred_prob.append(0)

In [ ]:
fi = dict(zip(X.columns,dt.feature_importances_))
for e in sorted(fi, key=fi.get, reverse = True):
    print(e,fi[e])

In [ ]:
y_test.value_counts()

In [ ]:
y_pred.sum()

In [83]:
print(); print(tumdegiskenler)
print(); print(metrics.confusion_matrix(y_test, y_pred))
print(); print(metrics.balanced_accuracy_score(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13188
           1       0.48      0.47      0.48       986

    accuracy                           0.93     14174
   macro avg       0.72      0.72      0.72     14174
weighted avg       0.93      0.93      0.93     14174


[[12672   516]
 [  518   468]]

0.7177592759045195


In [56]:
import lightgbm as ltb


model = ltb.LGBMClassifier(n_estimators = 500)
model.fit(X_train, y_train)
print(); print(model)
    
    # make predictions
expected_y  = y_test
predicted_y = model.predict(X_test)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [57]:
print(); print(metrics.classification_report(expected_y, predicted_y))
print(); print(metrics.confusion_matrix(expected_y, predicted_y))


              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98     15786
         1.0       0.99      0.97      0.98     15802

    accuracy                           0.98     31588
   macro avg       0.98      0.98      0.98     31588
weighted avg       0.98      0.98      0.98     31588


[[15617   169]
 [  466 15336]]
